## Iteration1 Feature Engineering with TensorFlow

# Plan na bazie MLE4.3.(5,6,7,8,9)
1. Floats change into categorical bins (MLE 4.3.1) - quntiles with same number of examples per bin? 
2. 

# Development

## Imports

In [22]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
#from tensorflow.keras import layers
from tensorflow import feature_column as fc
from sklearn.model_selection import train_test_split

print(f"BigQuery client library version: {bigquery.__version__}")
print(f"Tensorflow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")


BigQuery client library version: 3.34.0
Tensorflow version: 2.18.1
Pandas version: 2.2.3


## Data load

In [9]:
bq = bigquery.Client(project="af-finanzen")
query = """SELECT
      type
    , started
    , description
    , amount
    , fee
    , currency
    , state
    , account
    , month
    , CASE
      WHEN i1_true_label = 'PK Prezenty' THEN 'PK Rest'
      WHEN i1_true_label = 'Apt' THEN 'PK Kasia'
      ELSE i1_true_label
      END AS i1_true_label
FROM `af-finanzen.monatsabschluss.revolut_abrechnung`
"""
df = bq.query(query).to_dataframe()

# Train, test, validation split

In [11]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2) # val is 16% from df, train 74%
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1652 train examples
413 validation examples
517 test examples


In [12]:
train.to_csv('data/transak-i1-train.csv', encoding='utf-8', index=False)

In [13]:
val.to_csv('data/transak-i1-val.csv', encoding='utf-8', index=False)

In [14]:
test.to_csv('data/transak-i1-test.csv', encoding='utf-8', index=False)

In [ ]:
import itertools

def head(filename, lines=10):
  """Prints the first 'lines' lines of the file."""
  with open(filename, 'r') as f:
    for line in itertools.islice(f, lines):
      print(line, end='') # Use end='' to avoid extra newlines
  print(f"\n")

for file in ["data/transak-i1-train.csv", "data/transak-i1-val.csv", "data/transak-i1-test.csv"]:
  print(f"==> {file} <==")
  #head(file)


===== data/transak-i1-train.csv =====
===== data/transak-i1-val.csv =====
===== data/transak-i1-test.csv =====


## TF Dataset from Pandas DF

In [23]:
def df2dataset(df: pd.DataFrame, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop("i1_true_label")
    ds = tf.data.Dataset.from_tensor_slices(dict(df), labels)
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df), seed=42)
    # ds.map(lambda...) M3.2
    ds = ds.batch(batch_size=batch_size)
    return ds

In [24]:
train_ds = df2dataset(train)
val_ds = df2dataset(val, shuffle=True)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).